<a href="https://colab.research.google.com/github/SarvaniAnkala/Text_Summarisation/blob/main/NLP_TEXT_SUMMARISATION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets nltk torch pandas sklearn spacy sumy
import pandas as pd
import numpy as np
import torch
import nltk
from nltk.tokenize import sent_tokenize
from transformers import pipeline, BartForConditionalGeneration, BartTokenizer, PegasusForConditionalGeneration, PegasusTokenizer


  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [2]:
!pip install scikit-learn
import sklearn


In [3]:
# Load CompScholar Dataset
comp_df = pd.read_csv('CompScholar.csv')
print(comp_df.head())  # Display first few rows


   Paper Id                                        Paper Title  \
0         1  Multi-document Summarization via Deep Learning...   
1         2  NLP based Machine Learning Approaches for Text...   
2         3  Abstractive text summarization using LSTM-CNN ...   
3         4  DEXPERTS: Decoding-Time Controlled Text Genera...   
4         5     A Survey of Knowledge-enhanced Text Generation   

                                           Key Words  \
0  Multi-document summarization (MDS), Deep learn...   
1  Text summarization, Abstractive and extractive...   
2  Text mining . Abstractive text summarization ....   
3  Natural language generation, Controlled text g...   
4  text-to-text generation, natural language proc...   

                                            Abstract  \
0  Multi-document summarization (MDS) is an effec...   
1  Due to the plethora of data available today, t...   
2   Abstractive Text Summarization (ATS), which i...   
3  Despite recent advances in natural lang

In [4]:
!pip install datasets


In [20]:
from huggingface_hub import notebook_login
notebook_login()


In [21]:
from datasets import load_dataset
# Step 1: Load the Training Datasets (ArXiv & PubMed)
train_dataset_arxiv = load_dataset("ccdv/arxiv-summarization", split="train[:10%]")

In [22]:
# Load the PubMed summarization dataset
train_dataset_pubmed = load_dataset("ccdv/pubmed-summarization", split="train[:10%]")

In [23]:
import pandas as pd
from datasets import Dataset

df = pd.read_csv("CompScholar.csv")  # Ensure the file is in the correct directory
eval_dataset = Dataset.from_pandas(df)


In [24]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments
# Step 3: Choose the Model (T5, BART, or PEGASUS)
MODEL_NAME = "facebook/bart-large-cnn"  # You can also use "t5-small" or "google/pegasus-large"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)


In [25]:
# Step 4: Preprocessing Function
def preprocess_function(examples):
    inputs = [doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True, padding="max_length")
    labels = tokenizer(examples["abstract"], max_length=256, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [26]:
# Step 5: Tokenization
train_dataset_arxiv = train_dataset_arxiv.map(preprocess_function, batched=True)
train_dataset_pubmed = train_dataset_pubmed.map(preprocess_function, batched=True)

In [27]:
def preprocess_function(examples):
    inputs = [doc for doc in examples["Document"]]  # Use "Document" for articles
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True, padding="max_length")

    labels = tokenizer(examples["Summary"], max_length=256, truncation=True, padding="max_length")  # Use "Summary" for summaries
    model_inputs["labels"] = labels["input_ids"]

    return model_inputs


In [28]:
print(eval_dataset.column_names)


['Paper Id', 'Paper Title', 'Key Words', 'Abstract', 'Conclusion', 'Document', 'Paper Type', 'Summary', 'Topic', 'OCR', 'labels']


In [29]:
eval_dataset = eval_dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/371 [00:00<?, ? examples/s]

In [30]:
from datasets import concatenate_datasets

# Step 6: Merge Training Datasets
train_dataset = concatenate_datasets([train_dataset_arxiv, train_dataset_pubmed])


In [31]:
train_dataset = train_dataset.select(range(int(len(train_dataset) * 0.1)))

In [32]:
print(train_dataset)

Dataset({
    features: ['article', 'abstract', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 3229
})


In [33]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",  # Evaluate every few steps instead of every epoch
    save_strategy="steps",  # Save checkpoints every few steps to prevent loss of progress
    save_steps=1000,  # Save model every 1000 steps instead of every epoch
    per_device_train_batch_size=2,  # Reduce batch size to 2
    per_device_eval_batch_size=2,  # Reduce batch size to 2
    num_train_epochs=2,  # Reduce epochs (fewer passes over dataset)
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=250,  # Log more frequently for debugging
    gradient_accumulation_steps=8,  # Accumulate gradients for effective larger batch size
    fp16=True,  # Enable mixed precision (faster & less memory usage)
    push_to_hub=False,
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [34]:
# Step 8: Train the Model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)

<ipython-input-34-70f684338373>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: ankaladurgasarvani (ankaladurgasarvani-iiest-shibpur) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss
